# Electricity Markets

This example gradually builds up more and more complicated energy-only electricity markets in PyPSA, starting from a single bidding zone, going up to multiple bidding zones connected with transmission (NTCs) along with variable renewables and storage.

## Preliminaries

Here libraries are imported and data is defined.

In [13]:
import numpy as np

import pypsa

In [14]:
# marginal costs in EUR/MWh
marginal_costs = {"Wind": 0, "Hydro": 0, "Coal": 30, "Gas": 60, "Oil": 80}

# power plant capacities (nominal powers in MW) in each country (not necessarily realistic)
power_plant_p_nom = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000},
    "Mozambique": {
        "Hydro": 1200,
    },
    "Eswatini": {
        "Hydro": 600,
    },
}

# transmission capacities in MW (not necessarily realistic)
transmission = {
    "South Africa": {"Mozambique": 500, "Eswatini": 250},
    "Mozambique": {"Eswatini": 100},
}

# country electrical loads in MW (not necessarily realistic)
loads = {"South Africa": 42000, "Mozambique": 650, "Eswatini": 250}

## Single bidding zone with fixed load, one period

In this example we consider a single market bidding zone, South Africa.

The inelastic load has essentially infinite marginal utility (or higher than the marginal cost of any generator).

In [15]:
country = "South Africa"

n = pypsa.Network()

n.add("Bus", country)

for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
    )


n.add("Load", f"{country} load", bus=country, p_set=loads[country]);

Run optimisation to determine market dispatch:

In [16]:
n.optimize()

Index(['South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 4 primals, 9 duals
Objective: 1.29e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-c6addxtc has 9 rows; 4 cols; 12 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+03, 4e+04]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-9); columns 0(-4); elements 0(-12) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-c6addxtc
Model status        : Optimal
Objective value     :  1.2900000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-ahe0w_u3.sol


('ok', 'optimal')

Print the load active power (P) consumption:

In [17]:
n.loads_t.p

Load,South Africa load
snapshot,
now,42000.0


Print the generator active power (P) dispatch:

In [18]:
n.generators_t.p

Generator,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,
now,35000.0,3000.0,4000.0,-0.0


Print the clearing price, which corresponds  to gas:

In [19]:

n.buses_t.marginal_price

Bus,South Africa
snapshot,
now,60.0


## Two bidding zones connected by transmission, one period

In this example we have bidirectional lossless transmission capacity between two bidding zones. The power transfer is treated as controllable (like an A/NTC (Available/Net Transfer Capacity) or HVDC line). Note that in the physical grid, power would flow passively according to the network impedances.

In [20]:
n = pypsa.Network()

countries = ["Mozambique", "South Africa"]

for country in countries:
    n.add("Bus", country)

    for tech in power_plant_p_nom[country]:
        n.add(
            "Generator",
            f"{country} {tech}",
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
        )

    n.add("Load", f"{country} load", bus=country, p_set=loads[country])

    # add transmission as controllable Link
    if country not in transmission:
        continue

    for other_country in countries:
        if other_country not in transmission[country]:
            continue

        # NB: Link is by default unidirectional, so have to set p_min_pu = -1
        # to allow bidirectional (i.e. also negative) flow
        n.add(
            "Link",
            f"{country} - {other_country} link",
            bus0=country,
            bus1=other_country,
            p_nom=transmission[country][other_country],
            p_min_pu=-1,
        )

In [21]:
n.optimize()

Index(['South Africa - Mozambique link'], dtype='object', name='Link')
Index(['Mozambique', 'South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 14 duals
Objective: 1.26e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-8hx2965d has 14 rows; 6 cols; 19 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 4e+04]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-14); columns 0(-6); elements 0(-19) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-8hx2965d
Model status        : Optimal
Objective value     :  1.2600000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-ceuz0h5m.sol


('ok', 'optimal')

In [22]:
n.loads_t.p

Load,Mozambique load,South Africa load
snapshot,,
now,650.0,42000.0


In [23]:
n.generators_t.p

Generator,Mozambique Hydro,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,,
now,1150.0,35000.0,3000.0,3500.0,-0.0


The clearing price corresponds to hydro in Mozambique and gas in South Africa.

In [24]:
n.buses_t.marginal_price

Bus,Mozambique,South Africa
snapshot,,
now,-0.0,60.0


In [25]:
n.links_t.p0

Link,South Africa - Mozambique link
snapshot,
now,-500.0


The shadow prices of the links measure the inframarginal rent of the link, i.e. the difference between the marginal price of the two bidding zones.

In [26]:
n.links_t.mu_lower

Link
snapshot
now


## Three bidding zones connected by transmission, one period

In this example we have bidirectional lossless transmission capacity between three bidding zones. The power transfer is treated as controllable (like an A/NTC (Available/Net Transfer Capacity) or HVDC line). Note that in the physical grid, power would flow passively according to the network impedances.

In [27]:
n = pypsa.Network()

countries = ["Eswatini", "Mozambique", "South Africa"]

for country in countries:
    n.add("Bus", country)

    for tech in power_plant_p_nom[country]:
        n.add(
            "Generator",
            f"{country} {tech}",
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
        )

    n.add("Load", f"{country} load", bus=country, p_set=loads[country])

    if country not in transmission:
        continue

    for other_country in countries:
        if other_country not in transmission[country]:
            continue

        n.add(
            "Link",
            f"{country} - {other_country} link",
            bus0=country,
            bus1=other_country,
            p_nom=transmission[country][other_country],
            p_min_pu=-1,
        )

In [28]:
n.optimize()

Index(['Mozambique - Eswatini link', 'South Africa - Eswatini link',
       'South Africa - Mozambique link'],
      dtype='object', name='Link')
Index(['Eswatini', 'Mozambique', 'South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 9 primals, 21 duals
Objective: 1.24e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-caj4qi24 has 21 rows; 9 cols; 30 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 4e+04]
Presolving model
2 rows, 5 cols, 6 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-21); columns 0(-9); elements 0(-30) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-caj4qi24
Model status        : Optimal
Objective value     :  1.2450000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-2demthap.sol


('ok', 'optimal')

In [29]:
n.loads_t.p

Load,Eswatini load,Mozambique load,South Africa load
snapshot,,,
now,250.0,650.0,42000.0


In [30]:
n.generators_t.p

Generator,Eswatini Hydro,Mozambique Hydro,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,,,
now,600.0,1050.0,35000.0,3000.0,3250.0,-0.0


The clearing prices correspond to hydro in Eswatini and Mozambique, and gas in South Africa.

In [31]:
n.buses_t.marginal_price

Bus,Eswatini,Mozambique,South Africa
snapshot,,,
now,-0.0,-0.0,60.0


In [32]:
n.links_t.p0

Link,Mozambique - Eswatini link,South Africa - Eswatini link,South Africa - Mozambique link
snapshot,,,
now,-100.0,-250.0,-500.0


In [33]:
n.links_t.mu_lower

Link
snapshot
now


## Single bidding zone with price-sensitive industrial load, one period

In this example we consider a single market bidding zone, South Africa.

Now there is a large industrial load with a marginal utility which is low enough to interact with the generation marginal cost. See also the [demand elasticity example]().

In [34]:
country = "South Africa"

n = pypsa.Network()

n.add("Bus", country)

for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
    )

# standard high marginal utility consumers
n.add("Load", f"{country} load", bus=country, p_set=loads[country])

# add an industrial load as a negative-dispatch generator with marginal utility of 70 EUR/MWh for 8000 MW
n.add(
    "Generator",
    f"{country} industrial load",
    bus=country,
    p_max_pu=0,
    p_min_pu=-1,
    p_nom=8000,
    marginal_cost=70,
);

In [35]:
n.optimize()

Index(['South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 5 primals, 11 duals
Objective: 1.25e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-78ejhhbz has 11 rows; 5 cols; 15 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+03, 4e+04]
Presolving model
1 rows, 4 cols, 4 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-11); columns 0(-5); elements 0(-15) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-78ejhhbz
Model status        : Optimal
Objective value     :  1.2500000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-an0oiemd.sol


('ok', 'optimal')

In [36]:
n.loads_t.p

Load,South Africa load
snapshot,
now,42000.0


A look at the generator dispatch shows that only half of industrial load is served, because this maxes out gas; oil is too expensive with a marginal cost of 80 EUR/MWh compared to the industrial load marginal utility of 70 EUR/MWh.

In [37]:

n.generators_t.p

Generator,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil,South Africa industrial load
snapshot,,,,,
now,35000.0,3000.0,8000.0,-0.0,-4000.0


In [38]:
n.buses_t.marginal_price

Bus,South Africa
snapshot,
now,70.0


## Single bidding zone with fixed load, several periods

In this example we consider a single market bidding zone, South Africa.

We consider multiple time periods (labelled [0,1,2,3]) to represent variable wind generation.

In [39]:
country = "South Africa"

n = pypsa.Network()

n.set_snapshots(range(4))

n.add("Bus", country)

# availability (p_max_pu) is variable for wind
for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
        p_max_pu=([0.3, 0.6, 0.4, 0.5] if tech == "Wind" else 1),
    )

# load which varies over the snapshots
n.add(
    "Load",
    f"{country} load",
    bus=country,
    p_set=loads[country] + np.array([0, 1000, 3000, 4000]),
);

In [40]:
n.optimize()

Index(['South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 16 primals, 36 duals
Objective: 6.08e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-pe5l00ea has 36 rows; 16 cols; 48 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [9e+02, 5e+04]
Presolving model
4 rows, 12 cols, 12 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-36); columns 0(-16); elements 0(-48) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-pe5l00ea
Model status        : Optimal
Objective value     :  6.0820000000e+06
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-q6tu7cf0.sol


('ok', 'optimal')

In [41]:
n.loads_t.p

Load,South Africa load
snapshot,
0,42000.0
1,43000.0
2,45000.0
3,46000.0


In [42]:
n.generators_t.p

Generator,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,
0,35000.0,900.0,6100.0,-0.0
1,35000.0,1800.0,6200.0,-0.0
2,35000.0,1200.0,8000.0,800.0
3,35000.0,1500.0,8000.0,1500.0


In [43]:
n.buses_t.marginal_price

Bus,South Africa
snapshot,
0,60.0
1,60.0
2,80.0
3,80.0


## Single bidding zone with fixed load and storage, several periods

In this example we consider a single market bidding zone, South Africa.

We consider multiple time periods (labelled [0,1,2,3]) to represent variable wind generation. Storage is allowed to do price arbitrage to reduce oil consumption.

In [44]:
country = "South Africa"

n = pypsa.Network()

# snapshots labelled by [0,1,2,3]
n.set_snapshots(range(4))

n.add("Bus", country)

# p_max_pu is variable for wind
for tech in power_plant_p_nom[country]:
    n.add(
        "Generator",
        f"{country} {tech}",
        bus=country,
        p_nom=power_plant_p_nom[country][tech],
        marginal_cost=marginal_costs[tech],
        p_max_pu=([0.3, 0.6, 0.4, 0.5] if tech == "Wind" else 1),
    )

# load which varies over the snapshots
n.add(
    "Load",
    f"{country} load",
    bus=country,
    p_set=loads[country] + np.array([0, 1000, 3000, 4000]),
)

# storage unit to do price arbitrage
n.add(
    "StorageUnit",
    f"{country} pumped hydro",
    bus=country,
    p_nom=1000,
    max_hours=6,  # energy storage in terms of hours at full power
)

Index(['South Africa pumped hydro'], dtype='object')

In [45]:
n.optimize()

Index(['South Africa'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.03s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 28 primals, 64 duals
Objective: 6.05e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-71ttaalf has 64 rows; 28 cols; 95 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e+01, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [9e+02, 5e+04]
Presolving model
8 rows, 24 cols, 35 nonzeros  0s
3 rows, 7 cols, 9 nonzeros  0s
Dependent equations search running on 3 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
3 rows, 7 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-61); columns 7(-21); elements 9(-86)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          4     6.0460000000e+06 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-71ttaalf
Model status        : Optimal
Simplex   iterations: 4
Objective value 

('ok', 'optimal')

In [46]:
n.loads_t.p

Load,South Africa load
snapshot,
0,42000.0
1,43000.0
2,45000.0
3,46000.0


In [47]:
n.generators_t.p

Generator,South Africa Coal,South Africa Wind,South Africa Gas,South Africa Oil
snapshot,,,,
0,35000.0,900.0,6900.0,-0.0
1,35000.0,1800.0,7200.0,-0.0
2,35000.0,1200.0,8000.0,-0.0
3,35000.0,1500.0,8000.0,500.0


In [48]:
n.storage_units_t.p

StorageUnit,South Africa pumped hydro
snapshot,
0,-800.0
1,-1000.0
2,800.0
3,1000.0


In [49]:
n.storage_units_t.state_of_charge

StorageUnit,South Africa pumped hydro
snapshot,
0,800.0
1,1800.0
2,1000.0
3,-0.0


In [50]:
n.buses_t.marginal_price

Bus,South Africa
snapshot,
0,60.0
1,60.0
2,60.0
3,80.0
